# [3.3] Run Evals with Inspect

# Introduction

In this section, we'll learn how to use the Inspect library to run the evals you generated in [3.2] on current language models. At the end, you should have results giving evidence towards whether these models have the property you set out to measure.

For this, we will first need to familiarise ourselves with Inspect, which will be the majority of this chapter. This is a library we'll use to make running our evaluations easier.

Once we've done that we'll obtain **baselines** across different models. Baselines measure whether the model can understand all of your questions. For example, a model may not be capable enough to decide which answer is "power-seeking" even if it wanted to be power-seeking. Baselines will therefore give us the upper- and lower-bounds of the results we should expect to receive.

Then we will run our evaluation. Using Inspect, we'll be able to do a lot more than simply present the question and generate an answer. We'll be able to: vary whether the model uses chain-of-thought reasoning to come to a conclusion, vary whether the model critiques its own answer, vary whether a system prompt is provided to the model or not, and more. This will enable us to get collect more data, and to determine how model behavior changes in different contexts.

These exercises continue from chapter [3.2]. If you didn't manage to generate 300 questions, we've prepared 300 example model-generated questions for the **tendency to seek power** following on from chapter [3.1] and chapter [3.2]. Similarly to previous days, the solutions are written with tendency to seek power in mind. However, it shouldn't be too hard to change any texts or function names to align with the dataset you've created.

# Setup (Don't read just run)

In [1]:

try:
    import google.colab  # type: ignore

    IN_COLAB = True
except:
    IN_COLAB = False

import os, sys

chapter = "chapter3_llm_evals"
repo = "ARENA_3.0"

if IN_COLAB:
    # Install packages
    %pip install openai==0.28
    %pip install inspect_ai
    %pip install jaxtyping
    %pip install time


else:
    pass 

In [2]:
import os
import json
from typing import Literal
from inspect_ai import Task, eval, task
from inspect_ai.log import read_eval_log
#from utils import omit
from inspect_ai.model import ChatMessage
from inspect_ai.dataset import FieldSpec, json_dataset, Sample, example_dataset
from inspect_ai.solver._multiple_choice import (
    Solver,
    solver,
    Choices,
    TaskState,
    answer_options,
)
from inspect_ai.solver._critique import (
    DEFAULT_CRITIQUE_TEMPLATE,
    DEFAULT_CRITIQUE_COMPLETION_TEMPLATE,
)
from inspect_ai.scorer import model_graded_fact, match, answer, scorer, Scorer
from inspect_ai.scorer._metrics import accuracy, stderr
from inspect_ai.solver import (
    chain_of_thought,
    generate,
    self_critique,
    system_message,
    Generate,
)
from inspect_ai.model import (
    ChatMessageUser,
    ChatMessageSystem,
    ChatMessageAssistant,
    get_model,
)
import random
import jaxtyping
from itertools import product

# 1️⃣ Introduction to `inspect`

## Learning Objectives

- Understand how **solvers** and **scorers** function in the inspect framework.
- Familiarise yourself with **plans**.
- Understand how to write your own **solver** functions.
- Know how all of these classes come together to form a **Task** object.
- Finalize and run your evaluation on a series of models.

## Inspect
[Inspect](https://inspect.ai-safety-institute.org.uk/) is a library written by the UK AISI in order to streamline model evaluations. Inspect makes running eval experiments easier by:

- Providing functions for manipulating the input to the model ("solvers") and scoring the model's answers ("scorers").

- Automatically creating logging files to store useful information about the evaluations that we run.

- Providing a nice layout to view the results of our evals, so we don't have to look directly at model outputs (which can be messy and hard to read).


### Overview of Inspect


Inspect uses `Task` as the central object that contains all the information about the eval we'll run on the model, specifically it contains:

- The `dataset`of questions we will evaluate the model on.

- The `plan` that the evaluation will proceed along. This is a list of `Solver` functions. `Solver` functions can modify the evaluation questions and/or get the model to generate a response. A typical collection of solvers that forms a `plan` might look like:

    - A `chain_of_thought` function which will add a prompt after the evaluation question that instructs the model to use chain-of-thought before answering.

    - A `generate` function that calls the LLM API to generate a response to the question (which now also includes a chain-of-thought prompt).

    - A `self_critique` function that maintains the `ChatHistory` of the model so far, and adds a prompt asking the model to critique its own output.

    - Another `generate` solver which calls the LLM API to generate an output in response to the new `self_critique` prompt.

- The final component of a `Task` object is a `scorer` function, which specifies how to score the model's output.

The diagram below gives a rough sense of how these objects interact in `Inspect`


<img src="https://raw.githubusercontent.com/chloeli-15/ARENA_img/main/img/ch3-inspect-outline.png" width=900>

## Dataset

We will start by defining the dataset that goes into our `Task`. Inspect accepts datasets in CSV, JSON, and Hugging Face formats. It has built-in functions that read in a dataset from any of these sources and convert it into a dataset of `Sample` objects, which is the datatype that Inspect uses to store information about a question. A `Sample` stores the text of a question, and other information about that question in "fields" with standardized names. The 3 most important fields of the `Sample` object are:

- `input`: The input to the model. This consists of system and user messages formatted as chat messages (which Inspect stores as a `ChatMessage` object).

- `choices`: The multiple choice list of answer options. (This wouldn't be necessary in a non-multiple-choice evaluation).

- `target`: The "correct" answer output (or `answer_matching_behavior` in our context).

Additionally, the `metadata` field is useful for storing additional information about our questions or the experiment that do not fit into a standardized field (e.g. question categories, whether or not to conduct the evaluation with a system prompt etc.) See the [docs](https://inspect.ai-safety-institute.org.uk/datasets.html) on Inspect Datasets for more information.

<details> 
<summary>Aside: Longer ChatMessage lists</summary>
<br>
For more complicated evals, we're able to provide the model with an arbitrary length list of ChatMessages in the <code>input</code> field including: 
<ul> 
<li>Multiple user messages and system messages.</li>
&nbsp;
<li>Assistant messages that the model will believe it has produced in response to user and system messages. However we can write these ourselves to provide a synthetic conversation history (e.g. giving few-shot examples or conditioning the model to respond in a certain format).</li>
&nbsp;
<li>Tool call messages which can mimic the model's interaction with any tools that we give it. We'll learn more about tool use later in the agent evals section.</li>
</ul>
</details>

### Exercise - Write record_to_sample function

```c
Difficulty: 🔴⚪⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 5-10 minutes on this exercise.
```

You should fill in the `record_to_sample` function, which does the following: 
* Takes an item ("record") from your dataset.
* Maps the value in your item's custom fields to the standardized fields of `Sample` (e.g. `answer_matching_behavior` → `target`). 
* Returns a `Sample` object

Read the [field-mapping section](https://inspect.ai-safety-institute.org.uk/datasets.html#field-mapping) of the docs to see the syntax, then fill in the function. 

Your `input` should be a list of `ChatMessageSystem`, `ChatMessageUser`, `ChatMessageAssistant`, or `ChatMessageTool` messages.



In [15]:
def record_to_sample(record: dict) -> Sample:
    """
    Converts a item ("record") from the dataset into a Sample object, mapping the fields of the record to the fields of the Sample object.

    Args:
        record : A dictionary from the json dataset containing our evaluation questions

    Returns:
        Sample : A Sample object containing the information in the record
    """
    return Sample(
        input=[],
        target= "A",
        choices= [],
        metadata={},
    )

Now, we can convert our JSON dataset into a dataset of `Sample` objects compatible with `inspect` using its built-in `json_dataset()` function, with the following syntax (fill in the path to your dataset of questions):

In [16]:
eval_dataset = json_dataset(r"your/path/to/dataset/here.json", record_to_sample)

### An Example Evaluation

Below we can run and display the results of an example evaluation. A simple example task with a dataset, plan, and scorer is written below:

In [17]:
from inspect_ai.dataset import example_dataset


@task
def theory_of_mind() -> Task:
    return Task(
        dataset=example_dataset("theory_of_mind"),
        plan=[
            chain_of_thought(),
            generate(),
            self_critique(model="openai/gpt-4o-mini"),
        ],
        scorer=model_graded_fact(model="openai/gpt-4o-mini"),
    )

Now let's see what it looks like to run this example task through inspect using the `eval()` function:

In [ ]:
log = eval(
    theory_of_mind(),
    model="openai/gpt-4o-mini",
    limit=10,
    log_dir="./logs", 
)

### Exercise - Explore Inspect's Log Viewer
```c
Difficulty: 🔴⚪⚪⚪⚪
Importance: 🔵🔵⚪⚪⚪

You should spend up to 10-15 mins on this exercise
```

When you ran the above code, you should have seen an image like this:

<img src = "https://raw.githubusercontent.com/chloeli-15/ARENA_img/main/img/ch3-inspect-output-image.png" width = "600px">

which indicates that the eval ran correctly. Now we can view the results of this eval in Inspect's log viewer. To do this, run the code below. It will locally host a display of the results of the example evaluation at http://localhost:7575. You need to modify "`your_log_name`" in the --log-dir argument below, so that it matches the path shown after `Log:` in the display shown by `Inspect` (which will depend on the date and time). 

Run the code block below and then click through to **http://localhost:7575**.

For more information about the log viewer, you can read the docs [here](https://inspect.ai-safety-institute.org.uk/log-viewer.html).

<details><summary>Aside: Log names</summary> I'm fairly confident that when Inspect accesses logs, it utilises the name, date, and time information as a part of the way of accessing and presenting the logging data. Therefore, it seems that there is no easy way to rename log files to make them easier to access (I tried it, and Inspect didn't let me open them). </details>

In [ ]:
!inspect view --log-dir "your/log/path/here.json" --port 7575

<details><summary>Help: I'm running this from a remote machine and can't access the Log Viewer at localhost:7575</summary>

If you're running this from a remote machine in VScode and can't access localhost:7575, the first thing you should try is modifying the "Auto Forward Ports Source" VScode setting to "Process," as shown below.

![port forwarding](https://raw.githubusercontent.com/chloeli-15/ARENA_img/main/img/ch3-port-forwarding-picture.png)

If it still doesn't work, then **make sure you've changed this setting in all setting categories** (circled in blue in the image above).

If you're still having issues, then try a different localhost port, (i.e. change `--port 7575` to `--port 7576` or something).
</details>

# 2️⃣ Writing Solvers 

<!---
<details><summary>THINK THIS SHOULD BE DELETED; IF REIMPLEMENTING FUNCTIONS, THEN INSERT REF to EXISTING INSPECT FUNCTION IN THE EXERCISE ITSELF. </summary>Some useful built-in solvers include:

- `generate()`: Calls the LLM API to generate a response to the input.

- `prompt_template()`:  This modifies the user prompt by substituting the current prompt into the `{prompt}` placeholder within the specified template. Can also substitute variables defined in the `metadata` of any Sample object.

- `chain_of_thought()`: Standard chain of thought template with {prompt} substitution variable. The default template is: 

    ```python
    DEFAULT_COT_TEMPLATE="""
    {prompt} 
    
    Before answering, reason in a step-by-step manner as to get the right answer. Provide your answer at the end on its own line in the form "ANSWER: $ANSWER" (without quotes) where $ANSWER is the answer to the question."""
    ```

- `multiple_choice()`: Presents a multiple choice question (with a single correct answer) in the format:

    ```python
    SINGLE_ANSWER_TEMPLATE = r"""
    Answer the following multiple choice question. The entire content of your response should be of the following format: 'ANSWER: $LETTER' (without quotes) where LETTER is one of {letters}.

    {question}

    {choices}
    """.strip()
    ```
    where `{choices}` are formatted as ```"A) Choice 1\nB) Choice 2\nC) Choice 3"```. This solver then automatically calls generate to call the LLM API for a response to the multiple choice question.
</details> --->
Solvers are functions that can manipulate the input to the model, and get the model to generate output (see [docs here](https://inspect.ai-safety-institute.org.uk/solvers.html)). Rather than using `Inspect`'s built-in solvers, we will be re-implementing these solvers or writing our own custom solvers, since:

1. It gives us more flexibility to use solvers in ways that we want to.

2. It will let us define our own custom solvers if there's another idea we're interested in testing in our evals.  

The most fundamental solver in the `Inspect` library is the `generate()` solver. This solver makes a call to the LLM API and gets the model to generate a response. Then it automatically appends this response to the chat conversation history.

Here is the list of solvers that we will write:

* `prompt_template()`: This solver can add any information to the text of the question. For example, we could add an indication that the model should answer using chain-of-thought reasoning.

* `multiple_choice_template()`: This is similar to Inspect's [`multiple_choice`](https://inspect.ai-safety-institute.org.uk/solvers.html#built-in-solvers) function but our version won't call `generate()` automatically, it will just provide a template to format multiple choice questions.

* `system_message()`: This adds a system message to the chat history.

* `self-critique()`: This gets the model to critique its own answer.

* `make_choice()`: This tells the model to make a final decision, and calls `generate()`.

### TaskState

A solver function takes in `TaskState` as input and applies transformations to it (e.g. modifies the messages, tells the model to output). `TaskState` is the main data structure that `Inspect` uses to store information while interacting with the model. A `TaskState` object is initialized for each question, and stores relevant information about the chat history and model output as a set of attributes. There are 3 attributes of `TaskState` that solvers interact with (there are additional attributes that solvers do not interact with): 

1. `messages`
2. `user_prompt`
3. `output`

Read the Inspect description of `TaskState` objects [here](https://inspect.ai-safety-institute.org.uk/solvers.html#task-states-1).

[Diagram of TaskState objects and solvers and how they interact]

### Asyncio

When we write solvers, what we actually write are functions which return `solve` functions. The structure of our code generally looks like:

```python
def solver_name(args) -> Solver:
    async def solve(state : TaskState, generate : Generate) -> TaskState:
        #Modify Taskstate in some way
        return state
    return solve
```

You might be wondering why we're using the `async` keyword. This comes from the `asyncio` python library. We've written a colab which provides more information about the `asyncio` library [**here**](https://colab.research.google.com/drive/16c0RwamV1gQ6fQnJTL05hBLp4f6_GHg_).

If you want more information about how parallelism works in Inspect specifically, read [this section of the docs](https://inspect.ai-safety-institute.org.uk/parallelism.html#sec-parallel-solvers-and-scorers)!


## Solvers

Our solvers will be relatively atomic compared to what is theoretically possible in a single solver function (e.g. a solver will only add one message to the `ChatHistory` at a time, or get the model to generate a single response). Then, later on, we'll put our solvers into one big `plan` — which is a list of solvers that are executed sequentially — which will correspond to the evaluation that we ultimately run. This means that we want our solvers to do small tasks, so that it is easy to compose them together later. This also minimises how much code we'll have to rewrite, and enables us to make one change to e.g. our multiple-choice question layout, and have this change propagate through every evaluation that we run.


### Exercise - Write `prompt_template` function
```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵🔵⚪

You should spend up to 10-15 minutes on this exercise.
```

Write a solver function that checks if `state` has a `user_prompt`, and then modifies `state.user_prompt.text` according to a template string which contains `"{prompt}"` — where the original `user_prompt` will get placed in the template.

For example, if our original `user_prompt.text` from our dataset was: `"Do you want to take over the world?"`, and our template was `"{prompt} Think step by step."` Then the solver should change `user_prompt.text` to be `"Do you want to take over the world? Think step by step."`

You might find Inspect's [solver documentation](https://inspect.ai-safety-institute.org.uk/solvers.html), or even their [github source code](https://github.com/UKGovernmentBEIS/inspect_ai/tree/0d9f9c8d5b472ec4f247763fe301a5be5839a607/src/inspect_ai/solver) useful for this exercise (and the following exercise).

In [54]:
@solver
def prompt_template(template: str) -> Solver:
    """
    Returns a solve function which modifies the user prompt with the given template.

    Args:
        template : The template string to use to modify the user prompt. Must include {question} to be replaced with the original user

    Returns:
        solve : A solve function which modifies the user prompt with the given template
    """

    async def solve(state: TaskState, generate: Generate) -> TaskState:
        
        return state

    return solve

<details><summary>Hint</summary>

Use Python's `.format()` method for strings.

</details>

### Exercise - Write a multiple_choice_format solver
```c
Difficulty: 🔴⚪⚪⚪⚪
Importance: 🔵🔵⚪⚪⚪

You should spend up to 5-10 mins on this exercise.
```

This solver should check that `state` has both `user_prompt` and `choices` properties, and modify the `user_prompt` according to a template string which contains `{question}` and `{choices}`. Before putting `state.choices` into the template, you should use Inspect's `answer_options()` function to provide a layout for `state.choices`. You can read Inspect's code for the `answer_options()` function [here](https://github.com/UKGovernmentBEIS/inspect_ai/blob/ba03d981b3f5569f445ce988cc258578fc3f0b80/src/inspect_ai/solver/_multiple_choice.py#L37). 

In [21]:
@solver
def multiple_choice_format(template: str, **params: dict) -> Solver:
    """
    Returns a solve function which modifies the initial prompt to be in the format of a multiple choice question. Make sure that {question} and {choices} are in the template string, so that you can format those parts of the prompt.

    Args:
        template : The template string to use to modify the user prompt. Must include {question} and {choices} to be replaced with the original user prompt and the answer choices, respectively.

    Returns:
        solve : A solve function which modifies the user prompt with the given template
    """

    async def solve(state: TaskState, generate: Generate) -> TaskState:
        return state

    return solve

### Exercise - Write a make_choice solver
```c
Difficulty: 🔴⚪⚪⚪⚪
Importance: 🔵🔵⚪⚪⚪

You should spend up to 5-10 mins on this exercise.
```

This solver should add a user prompt to `state.messages` telling the model to make a final decision out of the options provided. 

In [45]:
@solver
def make_choice(prompt: str) -> Solver:
    """
    Returns a solve function which adds a user message at the end of the state.messages list with the given prompt.

    Args:
        prompt : The prompt to add to the user messages

    Returns:
        solve : A solve function which adds a user message with the given prompt to the end of the state.messages list
    """

    async def solve(state: TaskState, generate: Generate) -> TaskState:
        return state

    return solve


### Exercise - Write a system_message solver
```c
Difficulty:🔴🔴⚪⚪⚪
Importance:🔵🔵⚪⚪⚪

You should spend up to 10-15 mins on this exercise.
```

This solver should add a system message to `state.messages`. The system message should be inserted after the last system message in `state.messages`. You may want to write an `insert_system_message` function which finds the last system message in a list of `messages` and adds the given `system_message` after this.

In [23]:
@solver
def system_message(system_message: str, **params: dict) -> Solver:
    """
    Returns a solve function which inserts a system message with the given content into the state.messages list. The system message is inserted after the last system message in the list.

    Args:
        system_message : The content of the system message to insert into the state.messages list

    Returns:
        solve : A solve function which inserts a system message with the given content into the state.messages list
    """

    async def solve(state: TaskState, generate: Generate) -> TaskState:
        return state

    return solve



### Exercise - Implement self-critique 
```c
Difficulty: 🔴🔴🔴⚪⚪
Importance: 🔵🔵⚪⚪⚪

You should spend up to 20-25 mins on this exercise.
```

We will be re-implementing the built-in `self_critique` solver from `inspect`. This will enable us to modify the functionality of the solver more easily if we want to adapt how the model critiques its own thought processes. To see why we might want models to do self-critique, read [this paper](https://arxiv.org/abs/2310.08118). Self-critique is most useful when we're asking the model for a capability, but if your dataset contains more complex questions, self-critique might get the model to reach a different conclusion (there is a difference on the power-seeking example dataset between CoT + self-critique, and CoT alone).

<details><summary>How Inspect implements <code>self_critique</code></summary>

By default, Inspect will implement the `self_critique` function as follows:

1. Take the entirety of the model's output so far

2. Get the model to generate a criticism of the output, or repeat say "The original answer is fully correct" if it has no criticism to give. 

3. Append this criticism to the original output before any criticism was generated, but as a *user message* (**not** as an *assistant message*), so that the model believes that it is a user who is criticising its outputs, and not itself (I think this generally tend to make it more responsive to the criticism, but you may want to experiment with this and find out).

4. Get the model to respond to the criticism or repeat the answer fully if there is no criticism.

5. Continue the rest of the evaluation.

You may not want to implement it exactly this way, either way implementing this process would be a good start.

 </details>


In [24]:
@solver
def self_critique(
    model: str,
    critique_template: str | None,
    critique_completion_template: str | None,
    **params: dict,
) -> Solver:
    """
    Args:
        - model: The model we use to generate the self-critique

        - critique_template: This is the template for how you present the output of the model so far to itself, so that the model can *generate* the critique. This template should contain {question} and {completion} to be formatted.
        
        - critique_completion_template: This is the template for how you present the output and critique to the model, so that it can generate an updated response based on the critique. This template should include {question} and {completion} and {critique} to be formatted.

    This is built into Inspect, so if you're happy to make use their version without any modifications, you can just use Inspect's self_critique solver.
    """

    async def solve(state: TaskState, generate: Generate) -> TaskState:
        return state

    return solve

### Exercise (optional) - Write your own solvers
```c
Difficulty: 🔴🔴🔴⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 15-25 mins on this exercise.
```

Now write any solvers that you want to write. Some ideas for additional solvers you may want to include below:

- A "language" solver, which tells the model to respond in different languages (you could even get the model to provide the necessary translation for the question first).

- A solver which provides "fake" reasoning by the model where it has already reacted in certain ways, to see if this conditions the model response in different ways. You would have to use the ChatMessageUser and ChatMessageAssistant Inspect classes for this.

- A solver which tells the model to respond badly to the question. Then you could use the `self_critique` solver to see if the model will criticise and disagree with itself after the fact, when it can't see that it was told to respond poorly to the question in the first place.

- Some solvers which enable more advanced multi-turn evaluations. These would function similarly to `self-critique` solvers, where models interact with each other. For example, one model could encourage the model being evaluated to respond in a more power-seeking manner.

In [25]:
# Now you should be able to write any other solvers that might be useful for your specific evaluation task (if you can't already use any of the existing solvers to accomplish the evaluation you want to conduct). For example:
#
# A "language" template, which tells the model to respond in different languages (you could even get the model to provide the necessary translation first).
#
# A solver which provides "fake" reasoning by the model where it has already reacted in certain ways, to see if this conditions the model response in different ways. You would have to use the ChatMessageUser and ChatMessageAssistant Inspect classes for this
#
# A solver which tells the model to respond badly to the question, to see if the model will criticise and disagree with itself after the fact when prompted to respond as it wants to.
#
#Some solvers which enable more advanced multi-turn evaluations. These would function similarly to `self-critique` solvers, where models interact with each other. For example, one model could encourage the model being evaluated to respond in a more power-seeking manner.
#
# Any other ideas for solvers you might have.
#  Do so here!

### Exercise (optional) - Write prompts for your solvers
```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵🔵⚪

You should spend up to 10-20 mins on this exercise. 
```

Now write prompts for any of the solvers you're planning on including in your evaluation task. You might want to come back to this exercise later if you see the model reacting incorrectly to any of your prompting.

In [55]:
self_critique_critique_template = r""""""
self_critique_completion_template = r""""""
chain_of_thought_template = r""""""
multiple_choice_format_template = r""""""
make_choice_prompt = r""""""
system_message_prompt = r""""""

# 3️⃣ Writing Tasks and Evaluating

## Scorers

Scorers are what the inspect library uses to judge model output. We use them to determine whether the model's answer is the `target` answer. From the [inspect documentation](https://inspect.ai-safety-institute.org.uk/scorers.html):


><br>
>Scorers generally take one of the following forms:
>
>1. Extracting a specific answer out of a model’s completion output using a variety of heuristics.
>
>2. Applying a text similarity algorithm to see if the model’s completion is close to what is set out in the `target`.
>
>3. Using another model to assess whether the model’s completion satisfies a description of the ideal answer in `target`.
>
>4. Using another rubric entirely (e.g. did the model produce a valid version of a file format, etc.)
>
><br>

Since we are writing a multiple choice benchmark, for our purposes we can either use the `match()` scorer, or the `answer()` scorer. The docs for these scorer functions can be found [here](https://inspect.ai-safety-institute.org.uk/scorers.html#built-in-scorers) (and the code can be found [here](https://github.com/UKGovernmentBEIS/inspect_ai/blob/c59ce86b9785495338990d84897ccb69d46610ac/src/inspect_ai/scorer/_match.py) in the inspect_ai github). You should make sure you have a basic understanding of how the `match()` and `answer()` scorers work before you try to use them, so spend some time reading through the docs and code here.

<details><summary>Aside: some other scorer functions:</summary>
<br>
Some other scorer functions that would be useful in different evals are:

- `includes()` which determines whether the target appears anywhere in the model output.

- `pattern()` which extracts the answer from the model output using a regular expression (the `answer()` scorer is essentially a special case of `pattern()`, where Inspect has already provided the regex for you).

- `model_graded_qa()` which has another model assess whether the output is a correct answer based on guidance in `target`. This is especially useful for evals where the answer is more open-ended. Depending on what sort of questions you're asking here, you may want to change the prompt template that is fed to the grader model.

- `model_graded_fact()` which does the same as `model_graded_qa()`, but is specifically for whether the model has included relevant factual content when compared to an expert answer (this would be used mostly on a capability evaluation).
<br>
</details>

## Loading Datasets

Before we can finalize our evaluation task, we'll need to modify our `record_to_sample` function from earlier. There are two issues with the function we wrote:

- First of all, we didn't shuffle the order in which the choices are presented to the model. Your dataset may have been generated pre-shuffled, but if not, then you ought to be aware that the model has a bias towards the first answer. See for example [Narayanan & Kapoor: Does ChatGPT have a Liberal Bias?](https://www.aisnakeoil.com/p/does-chatgpt-have-a-liberal-bias) To mitigate this confounder, we should shuffle the choices (and answer) around when loading in the dataset.

<details><summary>Aside: Inspect's built-in shuffle function</summary> 

Inspect comes with a shuffle function for multiple choice questions. However, this function then automatically "unshuffles" *only some* of the model's responses back to how they are presented in your dataset.

I'm sure there are use cases for this, but in our case it's unnecessarily confusing to see the model reasoning about e.g. how much it thinks "option B" is the correct answer, and then Inspect telling you the model selected "option A" since its selection choice gets "unshuffled" but its reasoning doesn't. For this reason, and because it's not very hard to do, we will just build our own shuffler that doesn't do any unshuffling to any part of the output.</details>

- Secondly, our original `record_to_sample` function can only load questions with the corresponding system prompt. It is often useful to load the dataset without a system prompt. For example, this can be useful when we obtain baselines, since we may not want the model to answer "as though" it is in the context given by the system prompt, but we do want it to be aware of that context. We also may want to load without a system prompt when running an evaluation, so we can see how the model reacts to our questions when presented with a more generic "You are a harmless, helpful, honest AI Chat Assistant" system prompt (which we can then provide with our `system_message` solver). 

### Exercise - Write record_to_sample function with shuffling
```c
Difficulty: 🔴⚪⚪⚪⚪
Importance: 🔵🔵⚪⚪⚪

You should spend up to 5-10 mins on this exercise.
```

Take the record_to_sample function you wrote in section 1, and modify it so that it randomly shuffles the order of `choices`. Remember to correspondingly shuffle the `target` answer.

In [37]:
def record_to_sample_shuffle(record: dict) -> Sample:
    return Sample(
        input=[],
        target= "A",
        choices=[],
        metadata={},
    )

### Exercise - Write record_to_sample function to handle system prompts
```c
Difficulty: 🔴⚪⚪⚪⚪
Importance: 🔵🔵⚪⚪⚪

You should spend up to 10-15 mins on this exercise.
```

As mentioned above, there are two reasons we might not want the system prompt provided to the model in the usual way. Either:
- We might want the model not to have the specific context of the system prompt, and ask it to respond with a different system message (which we can do via the system_prompt solver we wrote in section 3).

- When we get baselines, we might want the model to provide an answer with a different system prompt, but we still want to provide the context from the original system prompt somewhere in the user prompt.

Write two version of the `record_to_sample` function which handles system prompts in each of these two ways. 

Remember to shuffle answers in this function as well.

In [38]:
def record_to_sample_no_system_prompt(record: dict) -> Sample:
    return Sample(
        input=[],
        target="A",
        choices=[],
        metadata={},
    )


def record_to_sample_system_prompt_as_context(record: dict) -> Sample:
    return Sample(
        input=[],
        target="A",
        choices=[],
        metadata={},
    )

Now let's wrap our `record_to_sample` functions into just one `record_to_sample` function, so that we can pass arguments to it and see the outcomes.

In [39]:
def record_to_sample(system: bool, system_prompt_as_context: bool):
    assert (
        not system or not system_prompt_as_context
    ), "ERROR: You can only set one of system or system_prompt_as_context to be True."

    def wrapper(record: dict) -> Sample:
        if system:
            return record_to_sample_shuffle(record)
        elif system_prompt_as_context:
            return record_to_sample_system_prompt_as_context(record)
        else:
            return record_to_sample_no_system_prompt(record)

    return wrapper

Now we can load our dataset in using these functions.

Just fill in `"your/path/to/eval_dataset.json"` below with the path to your dataset.

In [40]:
path_to_eval_dataset = r"your/path/to/dataset/here.json"

eval_dataset_system = json_dataset(
    path_to_eval_dataset,
    record_to_sample(system=True, system_prompt_as_context=False),
)
eval_dataset_no_system = json_dataset(
    path_to_eval_dataset,
    record_to_sample(system=False, system_prompt_as_context=False),
)
eval_dataset_system_as_context = json_dataset(
    path_to_eval_dataset,
    record_to_sample(system=False, system_prompt_as_context=True),
)
test_dataset = json_dataset(
    path_to_eval_dataset,
    record_to_sample(system=True, system_prompt_as_context=False),
)[0:10]
test_dataset_system_as_context = json_dataset(
    path_to_eval_dataset,
    record_to_sample(system=False, system_prompt_as_context=True),
)[0:10]

### Exercise - Implementing the benchmark_test task
```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵🔵⚪

You should spend up to 15-20 mins on this exercise. 
```

First build out a baseline test: This should give you a sense of whether the model can recognize what the target behavior option is. When running this test, you should use the `eval_dataset_system_as_context` dataset, since we won't necessarily want the model to answer *as though* it is placed in the context of the system prompt, but we still want it to be *aware* of that context.

A few notes:

- You need to ensure your prompts for benchmarking are finalized.

- You need to figure out how you're going to deal with Claude often refusing to answer unethically. (Possibly just accept you can't get a baseline here; if you push too hard Anthropic might ban you).

- You need to decide what path you want your evaluations to proceed along (i.e. what solvers you'll want to use). 

Build out the evaluation. You'll likely need to craft and test specialized prompts. We loaded in a `test_dataset_system_as_context` dataset above and you should use this to test how different models understand and respond to your prompts before you carry out the full evaluation.

<details><summary>Aside: Claude's refusals</summary>

Claude has a tendency to refuse to answer in certain "bad" ways. If you're trying to get a baseline for a model from the Claude series, you may not be able to get it to answer to a question like "Answer as though you're maximally evil." If you have a 2-choice dataset with "Yes/No" answers, you can get a decent enough estimate by asking it to "Answer as though you're maximally good" and then assuming it would answer in the opposite if it were asked to be "maximally evil." If you have more than 2 choices then you may have to avoid taking a "negative" baseline. 

</details>

In [ ]:
benchmark_template_CoT = r""""""

benchmark_template_MCQ = r""""""

benchmark_template_make_choice = r""""""

In [56]:
@task
def benchmark_eval(
    multiple_choice_template: str,
    chain_of_thought_template: str,
    make_choice_prompt: str,
    test_dataset: bool = True,
):
    """
    A task which carries out your benchmark evaluation.

    Args:
        multiple_choice_template : The template to use for the multiple choice evaluation

        chain_of_thought_template : The template to use for the chain of thought evaluation

        make_choice_prompt : The prompt to use for the make choice evaluation

        test_dataset : Whether to use the test dataset or not
    
    Returns:
        Task : A task which carries out your benchmark evaluation
    """
    return Task()


### Exercise - Using Inspect's `eval()` function
```c
Difficulty: 🔴⚪⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 5-10 mins on this exercise. 
```

Now use Inspect' `eval()` function and test that your task works. Try running it on `"gpt-4o-mini"` (since this model is cheap, and should be sufficient to flag any potential issues with your evaluation). This model should be capable of understanding what your questions are getting at, and if it doesn't manage to respond with the target behavior, you should *probably* update your prompts. Make sure to run this with `test_dataset=True` the first few times. To use `Inspect`'s eval function, you'll need to pass a `task`, a `model` and a `log_dir` (there are many other arguments `eval()` can take, but these are all that we'll need). 

In [ ]:
logs = eval(
    benchmark_eval(), # Task to evaluate (including the templates)
    model = "", # Model to evaluate
    log_dir = "./path/to/log/directory" # Add your log directory path here
    
)

### Exercise - Finalize and run your evaluation task
```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 10-15 mins on this exercise. 
```

Create one task which has the capacity to run every eval you may want to run. Do this using conditional logic to append solvers to `eval_plan` based on arguments in the task function. Then we'll be able to run our evals with one call to the `eval()` function by iterating over these different arguments.

Make sure to include assert statements in case two solvers are appended which shouldn't go together (for example, it doesn't make much sense to get the model to do `self_critique` if you haven't already gotten it to output some reasoning to critique).

In [61]:
@task
def power_seeking(system : bool, CoT : bool, self_critique_enabled : bool, self_critique_model : Literal["openai/gpt-4o-mini"] = "openai/gpt-4o-mini"):
    return Task()
    

### Exercise - Run your evaluation
```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 10-15 mins to code this exercise. (Running will take longer).
```
Now conduct your final evaluation. You can either use a series of nested `for` loops and conditionals to run the different versions of your task built above, or you could put them all the different parameters into one `itertools.product` object.

You should write this code so that it carries out your entire suite of evaluations on the model we want to evaluate. We could also run it on every model we want to evaluate all at once, however this would take a substantial amount of time, and the cost would be hard to predict. Before running it on larger models, test that everything works on `"openai/gpt-4o-mini"`, as this is by far the cheapest model currently available (and surprisingly capable)!

For ease of use later, you should put your actual evaluation logs in a different subfolder in `/logs/` than your baseline evaluation logs.

<details><summary> API Prices:</summary>

The pricings of models can be found [here for OpenAI](https://www.openai.com/api/pricing), and [here for Anthropic](https://www.anthropic.com/pricing#anthropic-api)

</details>


In [ ]:
eval_model= "openai/gpt-4o-mini"

for _system in [True,False]:
    for _CoT in [True,False]:
        for _self_critique_enabled in [True,False]:
            if _self_critique_enabled and not _CoT:
                pass
            else:
                logs = eval(
                    power_seeking(system = _system, CoT = _CoT, self_critique_enabled = _self_critique_enabled, self_critique_model = eval_model),
                    model = eval_model,
                    log_dir = r"path/to/log/directory"
                )

# 4️⃣ Bonus: Logs and plotting

Now that we've finished running our evaluations, we can look at the logs and plot the results. First, let's introduce the inspect log file.

### Intro to Inspect Eval Logs

We can read a log as an `EvalLog` object by using `Inspect`'s `read_eval_log()` function in the following syntax:

In [6]:
log = read_eval_log(r"path/to/log/file.json")

The `EvalLog` object has the following attributes, from the `Inspect` documentation:

<blockquote>

| Field       | Type     | Description |
| -------------- | -------------- | --------------------------------- |
| `status`      | `str`         | Status of evaluation (`"started"`, `"success"`, or `"error"`). |
| `eval` | `EvalSpec` | Top level eval details including task, model, creation time, dataset, etc. |
| `plan` | `EvalPlan` | List of solvers and model generation config used for the eval. |
| `samples` | `list[EvalSample]` | Each sample evaluated, including its input, output, target, and score. |
| `results` | `EvalResults` | Aggregate results computed by scorer metrics. |
| `stats` | `EvalStats` | Model usage statistics (input and output tokens). |
| `logging` | `list[LoggingMessage`] | Any logging messages we decided to include |
| `error` | `EvalError` | Error information (if `status == "error"`) including the traceback | 

</blockquote>

For long evals, you should not call `log.samples` without indexing, as it will return an *incredibly* long list. Before accessing any information about logs, you should always first check that the `status` of the log is `"success"`.

Uncomment and run the code below to see what these `EvalLog` attributes look like:


In [ ]:
print(log.status) 

#print(log.eval)

#print(log.plan)

#print(log.samples[0])

#print(log.results)

#print(log.stats)

#print(log.logging) 

#print(log.error)

### Exercise (optional) - Extract data from log files
```c
Difficulty:🔴🔴🔴🔴⚪
Importance: 🔵🔵⚪⚪⚪
```

Extract data from the log files that have been generated so far. Put this data in a pandas table. Then we'll write a plotting function to plot the results of our evaluation. Only do this exercise if you want to get familiar with plotting. Otherwise definitely look at the solution (which you wil llikely have to modify to )

In [ ]:
# Extract data here

### Exercise (optional) - Use plotly (and claude) to plot the results of your evaluation

In [ ]:
# Plot some cool graphs here